In [ ]:
import numpy as np
#for windows
import sys
sys.path.append(r'C:\Users\pedersen\gmid') # path to gmid repository
# ------
import matplotlib.pyplot as plt
from mosplot import load_lookup_table, LoadMosfet # make sure that mosplot can be found in the python path
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interactive_output, HBox, VBox
import matplotlib.ticker as ticker 

In [ ]:
def plot_data_vs_data(x_values, y_values, length, x_axis_name, y_axis_name='y', y_multiplier=1, log=False):
    x_values_flat = np.array(x_values).flatten()  
    y_values_flat = np.array(y_values, dtype=np.float64).flatten()  
    length_flat = np.array(length).flatten()

    unique_lengths = np.unique(length_flat)
    unique_lengths_in_micro = unique_lengths * 1e6
    
    def update_plot(selected_length, x_value=None, y_value=None):
        plt.figure(figsize=(8, 6))  # Ensure the plot is drawn fresh for each update

        if selected_length == "Show All":
            mask = np.ones_like(length_flat, dtype=bool)
        else:
            selected_length_in_micro = float(selected_length.replace(' μm', ''))
            tolerance = 0.1  
            mask = np.abs(length_flat * 1e6 - selected_length_in_micro) < tolerance  

        x_values_for_length = x_values_flat[mask]
        y_values_for_length = y_values_flat[mask] * y_multiplier  
        length_for_length = length_flat[mask] * 1e6  

        if selected_length == "Show All":
            for length_value in np.unique(length_for_length):
                mask_all = (length_for_length == length_value)
                plt.plot(x_values_for_length[mask_all], y_values_for_length[mask_all])

            min_length = np.min(unique_lengths_in_micro)
            max_length = np.max(unique_lengths_in_micro)
            plt.title(f'{y_axis_name} vs {x_axis_name} (Length from {min_length:.2f} μm to {max_length:.2f} μm)')

        else:
            plt.plot(x_values_for_length, y_values_for_length)
            plt.title(f'{y_axis_name} vs {x_axis_name} for {selected_length}')

        plt.xlabel(f'{x_axis_name}')
        plt.ylabel(f'{y_axis_name}')

        if log:
            plt.yscale('log')
            plt.gca().yaxis.set_major_locator(ticker.LogLocator(base=10, subs=[], numticks=10))
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'$10^{int(np.log10(x))}$'))
            plt.ylabel(f'{y_axis_name} (Log Base 10)') 
        
        if y_value is not None and x_value_widget.disabled:
            closest_index = np.abs(y_values_for_length - y_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='blue', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            print(f"The corresponding {x_axis_name} value for {y_axis_name} = {closest_y:.2f} is: {closest_x:.2f}")
        elif x_value is not None and y_value_widget.disabled:
            closest_index = np.abs(x_values_for_length - x_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            
            plt.scatter(closest_x, closest_y, color='red', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            print(f"The corresponding {y_axis_name} value for {x_axis_name} = {closest_x:.2f} is: {closest_y:.2f}")

        plt.grid(True)
        plt.legend()
        plt.show()

    dropdown_options = ["Show All"] + [f'{length:.2f} μm' for length in unique_lengths_in_micro]
    length_widget = widgets.Dropdown(
        options=dropdown_options,
        value=dropdown_options[0], 
        description='Select Length:',
    )
    
    x_value_widget = widgets.FloatText(
        value=np.mean(x_values_flat),
        description=f"Select {x_axis_name}:",
        disabled=False
    )
    
    y_value_widget = widgets.FloatText(
        value=None,
        description=f"Set {y_axis_name}:",
        disabled=True
    )
    
    select_x_or_y_widget = widgets.Checkbox(
        value=True,
        description=f"Select {x_axis_name} (uncheck for {y_axis_name})",
    )
    
    def toggle_x_or_y(change):
        if change['new']:
            x_value_widget.disabled = False
            y_value_widget.disabled = True
        else:
            x_value_widget.disabled = True
            y_value_widget.disabled = False
    
    select_x_or_y_widget.observe(toggle_x_or_y, names='value')

    output = interactive_output(update_plot, {
        'selected_length': length_widget, 
        'x_value': x_value_widget, 
        'y_value': y_value_widget
    })

    display(VBox([length_widget, select_x_or_y_widget, HBox([x_value_widget, y_value_widget]), output]))


In [ ]:
#pmos_lv_path = '/home/pedersen/gmid/models/gmoveridpypmos_gmid_final.npy'
#nmos_lv_path = '/home/pedersen/gmid/models/gmoveridpynmos_final_gmid.npy'

# Windows paths
pmos_lv_path = r'C:\Users\pedersen\Desktop\OpenDesingCourse\no_touch_files\gmoveridpypmos_gmid_final.npy'
nmos_lv_path =r'C:\Users\pedersen\Desktop\OpenDesingCourse\no_touch_files\gmoveridpynmos_final_gmid.npy'

In [4]:
lookup_table_pmos = load_lookup_table(pmos_lv_path)
lookup_table_nmos = load_lookup_table(nmos_lv_path)

In [5]:
nmos = LoadMosfet(lookup_table=lookup_table_nmos, mos="nmos", vsb=0.0, vds=0.6)
pmos = LoadMosfet(lookup_table=lookup_table_pmos, mos="pmos", vsb=0, vds=-0.6, vgs=(-1.2, -0.1))

In [13]:
vgs_nmos = nmos.extracted_table['vgs']
id_values = nmos.extracted_table['id']
plot_data_vs_data(vgs_nmos, id_values, nmos.extracted_table['lengths'], 'Vgs', 'Id[mu A]', y_multiplier=1e6, log=True)

In [7]:
id_values = nmos.extracted_table['id']
length = nmos.extracted_table['lengths']
gm = nmos.extracted_table['gm']
gds = nmos.extracted_table['gds']
gmro = gm/gds
gmid = gm/id_values

plot_data_vs_data(gmid, gm/gds, length,'gmid','gmro')

In [8]:
id_values = nmos.extracted_table['id']
width = nmos.extracted_table['width']
length = nmos.extracted_table['lengths']
gm = nmos.extracted_table['gm']
gmid = gm/id_values

plot_data_vs_data(gmid, id_values/width, length,'gmid','id/width', log = True)

In [14]:
id = 16e-6
gmid = 9.86
length = 1.3e-6
id_over_width = 0.64
width = id/id_over_width
print(f"Width = {width:}")

Width = 2.4999999999999998e-05
